> Essa análise inicia na [Parte 1](https://colab.research.google.com/drive/1mjPfIF7ds511-9AZH-kyEYTxa7WiboqL?usp=sharing)

# Parte 2 - Análise de perfis por região

In [ ]:
!pip install gdown
!gdown --id 18a4OdoBIuKaMsKzDusgrrrvFEBaVpwux
!gdown --id 1DOFjAZDr6zy4CBtLrm4ivwAtjdqzzDWh
!gdown --id 1d21EtDihdb7QCyoemkKZS-XLPB_z4Ggk

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=18a4OdoBIuKaMsKzDusgrrrvFEBaVpwux
To: /content/PERFIL_LOCALIDADE.csv
100% 9.54M/9.54M [00:00<00:00, 201MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1DOFjAZDr6zy4CBtLrm4ivwAtjdqzzDWh
To: /content/CANDIDATOS_MAIS_VOTADOS_2T.csv
100% 7.36k/7.36k [00:00<00:00, 16.9MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From:

Importações de bibliotecas

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
df_perfis = pd.read_csv("PERFIL_LOCALIDADE.csv")
df_perfis.head()

,Unnamed: 0,NR_ZONA,NR_SECAO,DS_GENERO,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,NR_LATITUDE,NR_LONGITUDE,NM_LOCAL_VOTACAO_ORIGINAL,NM_BAIRRO,DS_NOVA_FAIXA_ETARIA,NR_PERFIL
0,0,17,276,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,1.0,-7.239689,-35.864477,ESCOLA MUNICIPAL JOSELITA BRASILEIRO,VILA CABRAL DE SANTA TEREZINHA,Idosos,31.0
1,1,17,276,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,4.0,-7.239689,-35.864477,ESCOLA MUNICIPAL JOSELITA BRASILEIRO,VILA CABRAL DE SANTA TEREZINHA,Idosos,31.0
2,2,17,279,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,2.0,-7.275561,-35.914516,ESCOLA MUNICIPAL HENRIQUE GUILHERMINO BARBOSA,CATOLÉ DE ZÉ FERREIRA,Idosos,31.0
3,3,17,43,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,1.0,-7.229281,-35.876510,ESCOLA MUNICIPAL FÉLIX ARAÚJO,CATOLÉ,Idosos,31.0
4,4,17,46,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,2.0,-7.271481,-36.097464,EEEF RUBENS DUTRA SEGUNDO,DISTRITO DE CATOLÉ,Idosos,31.0


## Distribuição de abrangência por gênero

In [ ]:
ocorrencia_genero = df_perfis.groupby(["DS_GENERO", "NR_LATITUDE", "NR_LONGITUDE", "NM_BAIRRO"]).sum(numeric_only=True).reset_index()[["DS_GENERO", "NR_LATITUDE", "NR_LONGITUDE", "QT_ELEITORES_PERFIL", "NM_BAIRRO"]]
ocorrencia_genero

,DS_GENERO,NR_LATITUDE,NR_LONGITUDE,QT_ELEITORES_PERFIL,NM_BAIRRO
0,FEMININO,-7.343545,-36.087300,180.0,DISTRITO DE CATOLÉ
1,FEMININO,-7.311400,-35.813941,126.0,GALANTE
2,FEMININO,-7.306570,-35.782958,386.0,GALANTE
3,FEMININO,-7.306251,-35.774198,918.0,GALANTE
4,FEMININO,-7.305965,-35.782734,609.0,GALANTE
...,...,...,...,...,...
239,MASCULINO,-7.199940,-35.834648,43.0,SITIO MARINHO
240,MASCULINO,-7.199209,-35.895753,1475.0,PALMEIRA
241,MASCULINO,-7.185304,-35.905208,507.0,CUITES
242,MASCULINO,-7.185081,-35.985678,1399.0,DISTRITO DE SAO JOSE DA MATA


In [ ]:
contagem_ocorrencia_genero = ocorrencia_genero.groupby(['NR_LATITUDE', 'NR_LONGITUDE', "NM_BAIRRO"], as_index=False).apply(lambda group: group.nlargest(1, 'QT_ELEITORES_PERFIL'))
contagem_ocorrencia_genero

,,DS_GENERO,NR_LATITUDE,NR_LONGITUDE,QT_ELEITORES_PERFIL,NM_BAIRRO
0,122,MASCULINO,-7.343545,-36.087300,182.0,DISTRITO DE CATOLÉ
1,1,FEMININO,-7.311400,-35.813941,126.0,GALANTE
2,124,MASCULINO,-7.306570,-35.782958,419.0,GALANTE
3,125,MASCULINO,-7.306251,-35.774198,932.0,GALANTE
4,4,FEMININO,-7.305965,-35.782734,609.0,GALANTE
...,...,...,...,...,...,...
117,117,FEMININO,-7.199940,-35.834648,48.0,SITIO MARINHO
118,240,MASCULINO,-7.199209,-35.895753,1475.0,PALMEIRA
119,241,MASCULINO,-7.185304,-35.905208,507.0,CUITES
120,120,FEMININO,-7.185081,-35.985678,1534.0,DISTRITO DE SAO JOSE DA MATA


In [ ]:
fig1 = px.scatter_mapbox(contagem_ocorrencia_genero, lat='NR_LATITUDE', lon='NR_LONGITUDE', color='DS_GENERO', size='QT_ELEITORES_PERFIL',
                        center=dict(lat=-7.2307, lon=-35.8811), zoom=12,
                        hover_name='DS_GENERO', hover_data=['QT_ELEITORES_PERFIL', "NM_BAIRRO"],
                        title='Abrangência de gênero por eleitores aptos por local de votação',
                        mapbox_style='open-street-map',
                        color_discrete_sequence=["red", "blue"],
                        size_max=30)

fig1.update_layout(
    legend=dict(
        title='Gênero',
        orientation='h',
        itemsizing='constant'
    )
)

Com base na análise da distribuição de abrangência por gênero, podemos observar que a maior parte do eleitorado da cidade é feminino.

A observação já era esperada, tendo em vista que mais de 52,7% da população de Campina Grande é composta por pessoas do gênero feminino, de acordo com o Censo do IBGE de 2010.

Visualmente, não existe associação entre localidade e abrangência de eleitores por gênero, algo que também era esperado.

## Distribuição de abrangência por escolaridade

In [ ]:
ocorrencia_escolaridade = df_perfis.groupby(["DS_GRAU_ESCOLARIDADE", "NR_LATITUDE", "NR_LONGITUDE", "NM_BAIRRO"]).sum(numeric_only=True).reset_index()[["DS_GRAU_ESCOLARIDADE", "NR_LATITUDE", "NR_LONGITUDE", "QT_ELEITORES_PERFIL", "NM_BAIRRO"]]
ocorrencia_escolaridade

,DS_GRAU_ESCOLARIDADE,NR_LATITUDE,NR_LONGITUDE,QT_ELEITORES_PERFIL,NM_BAIRRO
0,ANALFABETO,-7.343545,-36.087300,48.0,DISTRITO DE CATOLÉ
1,ANALFABETO,-7.311400,-35.813941,21.0,GALANTE
2,ANALFABETO,-7.306570,-35.782958,50.0,GALANTE
3,ANALFABETO,-7.306251,-35.774198,81.0,GALANTE
4,ANALFABETO,-7.305965,-35.782734,66.0,GALANTE
...,...,...,...,...,...
971,SUPERIOR INCOMPLETO,-7.199940,-35.834648,3.0,SITIO MARINHO
972,SUPERIOR INCOMPLETO,-7.199209,-35.895753,147.0,PALMEIRA
973,SUPERIOR INCOMPLETO,-7.185304,-35.905208,39.0,CUITES
974,SUPERIOR INCOMPLETO,-7.185081,-35.985678,110.0,DISTRITO DE SAO JOSE DA MATA


In [ ]:
contagem_ocorrencia_escolaridade = ocorrencia_escolaridade.groupby(['NR_LATITUDE', 'NR_LONGITUDE', 'NM_BAIRRO'], as_index=False).apply(lambda group: group.nlargest(1, 'QT_ELEITORES_PERFIL'))
contagem_ocorrencia_escolaridade

,,DS_GRAU_ESCOLARIDADE,NR_LATITUDE,NR_LONGITUDE,QT_ELEITORES_PERFIL,NM_BAIRRO
0,244,ENSINO FUNDAMENTAL INCOMPLETO,-7.343545,-36.087300,130.0,DISTRITO DE CATOLÉ
1,245,ENSINO FUNDAMENTAL INCOMPLETO,-7.311400,-35.813941,74.0,GALANTE
2,246,ENSINO FUNDAMENTAL INCOMPLETO,-7.306570,-35.782958,259.0,GALANTE
3,247,ENSINO FUNDAMENTAL INCOMPLETO,-7.306251,-35.774198,527.0,GALANTE
4,248,ENSINO FUNDAMENTAL INCOMPLETO,-7.305965,-35.782734,420.0,GALANTE
...,...,...,...,...,...,...
117,605,ENSINO MÉDIO INCOMPLETO,-7.199940,-35.834648,26.0,SITIO MARINHO
118,484,ENSINO MÉDIO COMPLETO,-7.199209,-35.895753,900.0,PALMEIRA
119,485,ENSINO MÉDIO COMPLETO,-7.185304,-35.905208,255.0,CUITES
120,486,ENSINO MÉDIO COMPLETO,-7.185081,-35.985678,879.0,DISTRITO DE SAO JOSE DA MATA


In [ ]:
fig2 = px.scatter_mapbox(contagem_ocorrencia_escolaridade, lat='NR_LATITUDE', lon='NR_LONGITUDE', color='DS_GRAU_ESCOLARIDADE', size='QT_ELEITORES_PERFIL',
                        center=dict(lat=-7.2307, lon=-35.8811), zoom=12,
                        hover_name='DS_GRAU_ESCOLARIDADE', hover_data=['QT_ELEITORES_PERFIL', "NM_BAIRRO"],
                        title='Abrangência de nível de escolaridade dos eleitores aptos por local de votação',
                        mapbox_style='open-street-map',
                        size_max=30)

fig2.update_layout(
    legend=dict(
        title='Grau de Escolaridade',
        orientation='h',
        itemsizing='constant'
    )
)

Conseguimos observar na visualização acima que maior parte do eleitorado campinense apresenta **ENSINO MÉDIO COMPLETO**.

A região referente ao Sul da Zona Norte da cidade, que apresenta os melhores socio-indicadores segundo a [ANÁLISE RETROSPECTIVA E SITUAÇÃO ATUAL](https://drive.google.com/file/d/1PRwI37PUYDq_sS004OT4bTIvE3fpBCql/view) também compreende a área com maior incidência de eleitores com **ENSINO SUPERIOR COMPLETO**.

Já no equivalente aos eleitores com **ENSINO FUNDAMENTAL INCOMPLETO** observamos que a maior concentração se encontra nas regiões que obtiveram pior desempenho socioeconômico, geralmente áreas localizadas mais ao extremo da cidade.

## Distribuição de abrangência por faixa etária

In [ ]:
ocorrencia_faixa_etaria = df_perfis.groupby(["DS_NOVA_FAIXA_ETARIA", "NR_LATITUDE", "NR_LONGITUDE", "NM_BAIRRO"]).sum(numeric_only=True).reset_index()[["DS_NOVA_FAIXA_ETARIA", "NR_LATITUDE", "NR_LONGITUDE", "QT_ELEITORES_PERFIL", "NM_BAIRRO"]]
ocorrencia_faixa_etaria

,DS_NOVA_FAIXA_ETARIA,NR_LATITUDE,NR_LONGITUDE,QT_ELEITORES_PERFIL,NM_BAIRRO
0,Adultos,-7.343545,-36.087300,239.0,DISTRITO DE CATOLÉ
1,Adultos,-7.311400,-35.813941,150.0,GALANTE
2,Adultos,-7.306570,-35.782958,545.0,GALANTE
3,Adultos,-7.306251,-35.774198,1297.0,GALANTE
4,Adultos,-7.305965,-35.782734,828.0,GALANTE
...,...,...,...,...,...
358,Jovens,-7.199940,-35.834648,2.0,SITIO MARINHO
359,Jovens,-7.199209,-35.895753,212.0,PALMEIRA
360,Jovens,-7.185304,-35.905208,44.0,CUITES
361,Jovens,-7.185081,-35.985678,135.0,DISTRITO DE SAO JOSE DA MATA


In [ ]:
contagem_ocorrencia_faixa_etaria = ocorrencia_faixa_etaria.groupby(['NR_LATITUDE', 'NR_LONGITUDE'], as_index=False).apply(lambda group: group.nlargest(1, 'QT_ELEITORES_PERFIL'))
contagem_ocorrencia_faixa_etaria

,,DS_NOVA_FAIXA_ETARIA,NR_LATITUDE,NR_LONGITUDE,QT_ELEITORES_PERFIL,NM_BAIRRO
0,0,Adultos,-7.343545,-36.087300,239.0,DISTRITO DE CATOLÉ
1,1,Adultos,-7.311400,-35.813941,150.0,GALANTE
2,2,Adultos,-7.306570,-35.782958,545.0,GALANTE
3,3,Adultos,-7.306251,-35.774198,1297.0,GALANTE
4,4,Adultos,-7.305965,-35.782734,828.0,GALANTE
...,...,...,...,...,...,...
117,117,Adultos,-7.199940,-35.834648,60.0,SITIO MARINHO
118,118,Adultos,-7.199209,-35.895753,2144.0,PALMEIRA
119,119,Adultos,-7.185304,-35.905208,694.0,CUITES
120,120,Adultos,-7.185081,-35.985678,2186.0,DISTRITO DE SAO JOSE DA MATA


In [ ]:
fig3 = px.scatter_mapbox(contagem_ocorrencia_faixa_etaria, lat='NR_LATITUDE', lon='NR_LONGITUDE', color='DS_NOVA_FAIXA_ETARIA', size='QT_ELEITORES_PERFIL',
                        center=dict(lat=-7.2307, lon=-35.8811), zoom=12,
                        hover_name='DS_NOVA_FAIXA_ETARIA', hover_data=['QT_ELEITORES_PERFIL', "NM_BAIRRO"],
                        title='Abrangência de faixa etária dos eleitores aptos por local de votação',
                        mapbox_style='open-street-map',
                        size_max=30)

fig3.update_layout(
    legend=dict(
        title='Faixa etária',
        orientation='h',
        itemsizing='constant'
    )
)

Como esperado, a análise acima demonstra que grande parte do eleitorado de Campina Grande é formado por adultos (pessoas entre 19 e 59 anos), faixa que tem a obrigatoriedade do voto.

## Overview dos perfis existentes

In [ ]:
contagem_ocorrencia = df_perfis.groupby(["NR_PERFIL", "DS_GENERO", "DS_GRAU_ESCOLARIDADE", "DS_NOVA_FAIXA_ETARIA"]).sum(numeric_only=True).reset_index()[["NR_PERFIL", "DS_GENERO", "DS_GRAU_ESCOLARIDADE", "DS_NOVA_FAIXA_ETARIA", "QT_ELEITORES_PERFIL"]]

contagem_ocorrencia["CONTAGEM"] = contagem_ocorrencia["QT_ELEITORES_PERFIL"]
contagem_ocorrencia.drop(columns=["QT_ELEITORES_PERFIL"], inplace=True)
contagem_ocorrencia.head()

,NR_PERFIL,DS_GENERO,DS_GRAU_ESCOLARIDADE,DS_NOVA_FAIXA_ETARIA,CONTAGEM
0,0.0,FEMININO,ANALFABETO,Adultos,587.0
1,1.0,FEMININO,ANALFABETO,Idosos,1459.0
2,2.0,FEMININO,ANALFABETO,Jovens,5.0
3,3.0,FEMININO,ENSINO FUNDAMENTAL COMPLETO,Adultos,2267.0
4,4.0,FEMININO,ENSINO FUNDAMENTAL COMPLETO,Idosos,1907.0


In [ ]:
fig4 = px.bar(contagem_ocorrencia, x="NR_PERFIL", y="CONTAGEM", color="NR_PERFIL", title="Frequência de ocorrência de cada perfil dos eleitores aptos em toda a cidade", hover_name="NR_PERFIL")
fig4.show()

Podemos observar na visualização acima que a distribuição quantitativa da ocorrência dos perfis dos eleitores na cidade de Campina Grande apresenta uma distribuição bimodal, quando ordenados por índice. Isso significa que existem duas modas diferentes para o conjunto, evidenciadas pelos dois picos aparentes no gráfico.

Abaixo ranqueamos os perfis mais comuns em um top 10, para evidenciar as modas indicadas na visualização anterior.

In [ ]:
contagem_ocorrencia.sort_values(by=["CONTAGEM"], ascending=False)[0:10]

,NR_PERFIL,DS_GENERO,DS_GRAU_ESCOLARIDADE,DS_NOVA_FAIXA_ETARIA,CONTAGEM
9,9.0,FEMININO,ENSINO MÉDIO COMPLETO,Adultos,20823.0
33,33.0,MASCULINO,ENSINO MÉDIO COMPLETO,Adultos,17350.0
36,36.0,MASCULINO,ENSINO MÉDIO INCOMPLETO,Adultos,13514.0
12,12.0,FEMININO,ENSINO MÉDIO INCOMPLETO,Adultos,12537.0
30,30.0,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,Adultos,11319.0
6,6.0,FEMININO,ENSINO FUNDAMENTAL INCOMPLETO,Adultos,9849.0
21,21.0,FEMININO,SUPERIOR INCOMPLETO,Adultos,7491.0
18,18.0,FEMININO,SUPERIOR COMPLETO,Adultos,7445.0
45,45.0,MASCULINO,SUPERIOR INCOMPLETO,Adultos,6683.0
7,7.0,FEMININO,ENSINO FUNDAMENTAL INCOMPLETO,Idosos,5777.0


Observamos que o ranking acima condiz com as análises realizadas por cada uma das características de perfil anteriormente realizadas. Por exemplo, as três variáveis que se destacaram no perfil 9 (Top 1), condiz com as visualizações observadas anteriormente, dado que o gênero FEMININO foi o mais abrangente, o grau de escolaridade que mais se destacou foi ENSINO MÉDIO COMPLETO e a faixa etária mais frequente foi ADULTOS, todos esses valores são descritores do perfil 9.

## Distribuição dos perfis pela cidade

In [ ]:
ocorrencia_perfil_local = df_perfis.groupby(['NR_LATITUDE', 'NR_LONGITUDE', "NM_BAIRRO"], as_index=False).apply(lambda group: group.nlargest(1, 'QT_ELEITORES_PERFIL'))
ocorrencia_perfil_local.head()

,,Unnamed: 0,NR_ZONA,NR_SECAO,DS_GENERO,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,NR_LATITUDE,NR_LONGITUDE,NM_LOCAL_VOTACAO_ORIGINAL,NM_BAIRRO,DS_NOVA_FAIXA_ETARIA,NR_PERFIL
0,48099,76075,17,47,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,10.0,-7.343545,-36.087300,ESCOLA MUNICIPAL MANOEL JOAQUIM DE AGUIAR - UN...,DISTRITO DE CATOLÉ,Adultos,30.0
1,5267,6034,17,249,MASCULINO,ENSINO MÉDIO INCOMPLETO,7.0,-7.311400,-35.813941,ESCOLA MUNICIPAL PROFESSOR ARIEL,GALANTE,Adultos,36.0
2,49860,79171,17,226,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,14.0,-7.306570,-35.782958,ESCOLA MUNICIPAL IRACEMA PIMENTEL,GALANTE,Adultos,30.0
3,19435,26163,17,359,FEMININO,ENSINO MÉDIO INCOMPLETO,32.0,-7.306251,-35.774198,EEEFM ISABEL RODRIGUES DE MELO,GALANTE,Adultos,12.0
4,25397,33571,17,274,FEMININO,ENSINO FUNDAMENTAL INCOMPLETO,27.0,-7.305965,-35.782734,EEEFM MONSENHOR SALES,GALANTE,Adultos,6.0


In [ ]:
ocorrencia_perfil_local["RÓTULO"] = ocorrencia_perfil_local.DS_NOVA_FAIXA_ETARIA + " " + ocorrencia_perfil_local.DS_GENERO  + " " + ocorrencia_perfil_local.DS_GRAU_ESCOLARIDADE

In [ ]:
fig2 = px.scatter_mapbox(ocorrencia_perfil_local, lat='NR_LATITUDE', lon='NR_LONGITUDE', color='RÓTULO', size='QT_ELEITORES_PERFIL',
                        center=dict(lat=-7.2307, lon=-35.8811), zoom=12,
                        hover_name='RÓTULO', hover_data=['QT_ELEITORES_PERFIL'],
                        title='Perfil mais abrangente por local de votação',
                        mapbox_style='open-street-map',
                        size_max=30)

fig2.update_layout(
    legend=dict(
        title='PERFIL',
        orientation='h',
        itemsizing='constant'
    )
)

Para o perfil ADULTOS SUPERIOR COMPLETO com ambos os gêneros, FEMININO e MASCULINO, a maior concentração do eleitorado se dá na região central da cidade, condizente com as áreas de ponderação com maiores indicadores de melhor desempenho socioeconômicos.

Já nas regiões mais distritais e com os maiores indicadores de pior desempenho, o perfil que mais se destacou foi ADULTOS ENSINO FUNDAMENTAL INCOMPLETO para ambos os gêneros.

# Perfis mais abrangentes por local de votação e candidatos que venceram nesses locais

## Unindo dados das análises considerando o perfil abrangente por local de votação

In [ ]:
ganhos_1t = pd.read_csv("CANDIDATOS_MAIS_VOTADOS_1T.csv")
ganhos_2t = pd.read_csv("CANDIDATOS_MAIS_VOTADOS_2T.csv")

In [ ]:
ganhos_1t.drop(columns=["Unnamed: 0", "Unnamed: 1"], inplace=True)
ganhos_2t.drop(columns=["Unnamed: 0", "Unnamed: 1"], inplace=True)

In [ ]:
ganhos_1t.head()

,NM_VOTAVEL,NR_LATITUDE,NR_LONGITUDE,QT_VOTOS,NM_BAIRRO
0,LULA,-7.343545,-36.087300,288.0,DISTRITO DE CATOLÉ
1,LULA,-7.311400,-35.813941,206.0,GALANTE
2,LULA,-7.306570,-35.782958,632.0,GALANTE
3,LULA,-7.306251,-35.774198,1402.0,GALANTE
4,LULA,-7.305965,-35.782734,892.0,GALANTE


In [ ]:
ocorrencia_perfil_local.head()

,,Unnamed: 0,NR_ZONA,NR_SECAO,DS_GENERO,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,NR_LATITUDE,NR_LONGITUDE,NM_LOCAL_VOTACAO_ORIGINAL,NM_BAIRRO,DS_NOVA_FAIXA_ETARIA,NR_PERFIL,RÓTULO
0,48099,76075,17,47,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,10.0,-7.343545,-36.087300,ESCOLA MUNICIPAL MANOEL JOAQUIM DE AGUIAR - UN...,DISTRITO DE CATOLÉ,Adultos,30.0,Adultos MASCULINO ENSINO FUNDAMENTAL INCOMPLETO
1,5267,6034,17,249,MASCULINO,ENSINO MÉDIO INCOMPLETO,7.0,-7.311400,-35.813941,ESCOLA MUNICIPAL PROFESSOR ARIEL,GALANTE,Adultos,36.0,Adultos MASCULINO ENSINO MÉDIO INCOMPLETO
2,49860,79171,17,226,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,14.0,-7.306570,-35.782958,ESCOLA MUNICIPAL IRACEMA PIMENTEL,GALANTE,Adultos,30.0,Adultos MASCULINO ENSINO FUNDAMENTAL INCOMPLETO
3,19435,26163,17,359,FEMININO,ENSINO MÉDIO INCOMPLETO,32.0,-7.306251,-35.774198,EEEFM ISABEL RODRIGUES DE MELO,GALANTE,Adultos,12.0,Adultos FEMININO ENSINO MÉDIO INCOMPLETO
4,25397,33571,17,274,FEMININO,ENSINO FUNDAMENTAL INCOMPLETO,27.0,-7.305965,-35.782734,EEEFM MONSENHOR SALES,GALANTE,Adultos,6.0,Adultos FEMININO ENSINO FUNDAMENTAL INCOMPLETO


In [ ]:
colunas_comuns = list(filter(lambda x: x in ganhos_1t.columns, ganhos_2t.columns))
colunas_comuns.remove("NM_VOTAVEL")
colunas_comuns.remove("QT_VOTOS")
colunas_comuns

['NR_LATITUDE', 'NR_LONGITUDE', 'NM_BAIRRO']

In [ ]:
merge = pd.merge(ganhos_1t, ganhos_2t, how='outer',
                  left_on=colunas_comuns,
                  right_on=colunas_comuns)

merge.dropna(inplace=True)

In [ ]:
merge.rename(columns={
    "NM_VOTAVEL_x": "NM_VOTAVEL_1T",
    "NM_VOTAVEL_y": "NM_VOTAVEL_2T",
    "QT_VOTOS_x": "QT_VOTOS_1T",
    "QT_VOTOS_y": "QT_VOTOS_2T"

}, inplace=True)

In [ ]:
merge.head()

,NM_VOTAVEL_1T,NR_LATITUDE,NR_LONGITUDE,QT_VOTOS_1T,NM_BAIRRO,NM_VOTAVEL_2T,QT_VOTOS_2T
0,LULA,-7.343545,-36.087300,288.0,DISTRITO DE CATOLÉ,LULA,323.0
1,LULA,-7.311400,-35.813941,206.0,GALANTE,LULA,234.0
2,LULA,-7.306570,-35.782958,632.0,GALANTE,LULA,644.0
3,LULA,-7.306251,-35.774198,1402.0,GALANTE,LULA,1432.0
4,LULA,-7.305965,-35.782734,892.0,GALANTE,LULA,923.0


In [ ]:
colunas_comuns = list(filter(lambda x: x in ocorrencia_perfil_local.columns, merge.columns))
colunas_comuns

['NR_LATITUDE', 'NR_LONGITUDE', 'NM_BAIRRO']

In [ ]:
merge = pd.merge(merge, ocorrencia_perfil_local, how='outer',
                  left_on=colunas_comuns,
                  right_on=colunas_comuns)

merge.dropna(inplace=True)

In [ ]:
merge.head()

,NM_VOTAVEL_1T,NR_LATITUDE,NR_LONGITUDE,QT_VOTOS_1T,NM_BAIRRO,NM_VOTAVEL_2T,QT_VOTOS_2T,Unnamed: 0,NR_ZONA,NR_SECAO,DS_GENERO,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,NM_LOCAL_VOTACAO_ORIGINAL,DS_NOVA_FAIXA_ETARIA,NR_PERFIL,RÓTULO
0,LULA,-7.343545,-36.087300,288.0,DISTRITO DE CATOLÉ,LULA,323.0,76075.0,17.0,47.0,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,10.0,ESCOLA MUNICIPAL MANOEL JOAQUIM DE AGUIAR - UN...,Adultos,30.0,Adultos MASCULINO ENSINO FUNDAMENTAL INCOMPLETO
1,LULA,-7.311400,-35.813941,206.0,GALANTE,LULA,234.0,6034.0,17.0,249.0,MASCULINO,ENSINO MÉDIO INCOMPLETO,7.0,ESCOLA MUNICIPAL PROFESSOR ARIEL,Adultos,36.0,Adultos MASCULINO ENSINO MÉDIO INCOMPLETO
2,LULA,-7.306570,-35.782958,632.0,GALANTE,LULA,644.0,79171.0,17.0,226.0,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,14.0,ESCOLA MUNICIPAL IRACEMA PIMENTEL,Adultos,30.0,Adultos MASCULINO ENSINO FUNDAMENTAL INCOMPLETO
3,LULA,-7.306251,-35.774198,1402.0,GALANTE,LULA,1432.0,26163.0,17.0,359.0,FEMININO,ENSINO MÉDIO INCOMPLETO,32.0,EEEFM ISABEL RODRIGUES DE MELO,Adultos,12.0,Adultos FEMININO ENSINO MÉDIO INCOMPLETO
4,LULA,-7.305965,-35.782734,892.0,GALANTE,LULA,923.0,33571.0,17.0,274.0,FEMININO,ENSINO FUNDAMENTAL INCOMPLETO,27.0,EEEFM MONSENHOR SALES,Adultos,6.0,Adultos FEMININO ENSINO FUNDAMENTAL INCOMPLETO


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [ ]:
merge['NR_VOTAVEL_1T'] = label_encoder.fit_transform(merge['NM_VOTAVEL_1T'])
merge['NR_VOTAVEL_2T'] = label_encoder.fit_transform(merge['NM_VOTAVEL_2T'])
merge['NR_BAIRRO'] = label_encoder.fit_transform(merge['NM_BAIRRO'])
merge['NR_GENERO'] = label_encoder.fit_transform(merge['DS_GENERO'])
merge['NR_GRAU_ESCOLARIDADE'] = label_encoder.fit_transform(merge["DS_GRAU_ESCOLARIDADE"])
merge['NR_LOCAL_VOTACAO'] = label_encoder.fit_transform(merge['NM_LOCAL_VOTACAO_ORIGINAL'])
merge['NR_FAIXA_ETARIA'] = label_encoder.fit_transform(merge['DS_NOVA_FAIXA_ETARIA'])

merge.head()

,NM_VOTAVEL_1T,NR_LATITUDE,NR_LONGITUDE,QT_VOTOS_1T,NM_BAIRRO,NM_VOTAVEL_2T,QT_VOTOS_2T,Unnamed: 0,NR_ZONA,NR_SECAO,...,DS_NOVA_FAIXA_ETARIA,NR_PERFIL,RÓTULO,NR_VOTAVEL_1T,NR_VOTAVEL_2T,NR_BAIRRO,NR_GENERO,NR_GRAU_ESCOLARIDADE,NR_LOCAL_VOTACAO,NR_FAIXA_ETARIA
0,LULA,-7.343545,-36.087300,288.0,DISTRITO DE CATOLÉ,LULA,323.0,76075.0,17.0,47.0,...,Adultos,30.0,Adultos MASCULINO ENSINO FUNDAMENTAL INCOMPLETO,1,1,19,1,0,87,0
1,LULA,-7.311400,-35.813941,206.0,GALANTE,LULA,234.0,6034.0,17.0,249.0,...,Adultos,36.0,Adultos MASCULINO ENSINO MÉDIO INCOMPLETO,1,1,22,1,2,96,0
2,LULA,-7.306570,-35.782958,632.0,GALANTE,LULA,644.0,79171.0,17.0,226.0,...,Adultos,30.0,Adultos MASCULINO ENSINO FUNDAMENTAL INCOMPLETO,1,1,22,1,0,78,0
3,LULA,-7.306251,-35.774198,1402.0,GALANTE,LULA,1432.0,26163.0,17.0,359.0,...,Adultos,12.0,Adultos FEMININO ENSINO MÉDIO INCOMPLETO,1,1,22,0,2,34,0
4,LULA,-7.305965,-35.782734,892.0,GALANTE,LULA,923.0,33571.0,17.0,274.0,...,Adultos,6.0,Adultos FEMININO ENSINO FUNDAMENTAL INCOMPLETO,1,1,22,0,0,35,0


In [ ]:
merge.drop(columns=['NM_VOTAVEL_1T', 'NM_VOTAVEL_2T', 'NM_BAIRRO', 'DS_GENERO', 'DS_GRAU_ESCOLARIDADE', 'NM_LOCAL_VOTACAO_ORIGINAL', 'DS_NOVA_FAIXA_ETARIA', 'RÓTULO'], inplace=True)

## Teste Qui-Quadrado para observar dependência entre as variáveis

Como a maioria das nossas variáveis de análise principais são categóricas, encontramos um impasse no cálculo das correlações pelos métodos tradicionais (Pearson, Spearman e Kendall). Portanto, para avaliar se as variáveis apresentam alguma associação entre si, utilizaremos o teste Qui-Quadrado, com um _p-valor=0.05_, para que seja realizado o teste de hipóteses considerando as seguintes hipóteses nula e alternativa:

- **H0: As variáveis são independentes (não há associação)**
- **H1: As variáveis são dependentes (há associação)**

Observamos as colunas duas a duas, gerando todas as combinações possíveis.

In [ ]:
import scipy.stats as stats

filtro = merge.drop(columns=['QT_VOTOS_1T', 'QT_VOTOS_2T', 'QT_ELEITORES_PERFIL', 'Unnamed: 0', 'NR_SECAO', 'NR_ZONA', 'NR_LATITUDE', 'NR_LONGITUDE']).dropna()
analisados = []

for c1 in filtro.columns:
    for c2 in filtro.columns:
        if c1 == c2 or (c1, c2) in analisados or (c2, c1) in analisados:
            continue

        analisados.append((c1, c2))
        try:
            chi2_stat, p_val, dof, expected = stats.chi2_contingency(filtro[[c1, c2]].values)

            if p_val < 0.05:
                print(f"Existe uma associação entre as variáveis {c1} e {c2}. [p_valor = {p_val}]")
            else:
                print(f"Não há evidência de associação entre as variáveis {c1} e {c2}. [p_valor = {p_val}]")
        except:
            continue


Não há evidência de associação entre as variáveis NR_PERFIL e NR_VOTAVEL_1T. [p_valor = 0.9991021497380144]
Não há evidência de associação entre as variáveis NR_PERFIL e NR_VOTAVEL_2T. [p_valor = 0.9863291382087457]
Existe uma associação entre as variáveis NR_PERFIL e NR_BAIRRO. [p_valor = 2.2830694503878132e-126]
Não há evidência de associação entre as variáveis NR_PERFIL e NR_GENERO. [p_valor = 1.0]
Não há evidência de associação entre as variáveis NR_PERFIL e NR_GRAU_ESCOLARIDADE. [p_valor = 0.9296319634483919]
Existe uma associação entre as variáveis NR_PERFIL e NR_LOCAL_VOTACAO. [p_valor = 1.2173905341498584e-300]
Existe uma associação entre as variáveis NR_PERFIL e NR_FAIXA_ETARIA. [p_valor = 6.2801120275540075e-19]
Existe uma associação entre as variáveis NR_VOTAVEL_1T e NR_BAIRRO. [p_valor = 0.028971829547198916]
Não há evidência de associação entre as variáveis NR_VOTAVEL_1T e NR_GRAU_ESCOLARIDADE. [p_valor = 0.9999998798208501]
Existe uma associação entre as variáveis NR_VOTA

Do relatório acima, podemos destacar algumas associações interessantes:

- Há uma associação entre NR_VOTAVEL_1T, NR_VOTAVEL_2T com NR_BAIRRO, NR_LOCAL_VOTACAO, o que pode indicar que existe uma certa vinculação entre o lugar em que o voto foi depositado e para quem esse voto foi dado.

- Existe associação entre NR_PERFIL e o NR_BAIRRO, tal qual com o NR_LOCAL_VOTACAO, o que vem a indicar possível vínculo entre o perfil do usuário e o local de proximidade de votação do mesmo.

- Não há associação entre a variável NR_PERFIL e NR_VOTAVEL_1T ou NR_VOTAVEL_2T, o que indica que o perfil do eleitor não mostrou vínculo direto com os candidatos - provavelmente pelo fato de haverem votos depositados para ambos os candidatos vindos da maioria dos perfis.

## Teste Eta Quadrado Parcial

O teste do Eta ao Quadrado serve para refletir matemáticamente qual é o efeito de uma variável sobre a outra, se assemelhando ao intuíto teste do R2 que vimos durante a disciplina, mas que pode ser aplicado a variáveis categóricas. Dessa maneira, vamos executar o teste para as combinações de colunas que temos e observar o grau de efeito de uma variável sobre a outra.

In [ ]:
analisados = []

for coluna1 in filtro.columns:
    for coluna2 in filtro.columns:
        if coluna1 == coluna2 or (coluna1, coluna2) in analisados or (coluna2, coluna1) in analisados:
            continue

        analisados.append((coluna1, coluna2))
        modelo = ols(f'{coluna1} ~ {coluna2}', data=merge).fit()
        tabela_anova = sm.stats.anova_lm(modelo)
        eta_squared_partial = tabela_anova['sum_sq'][0] / (tabela_anova['sum_sq'][0] + tabela_anova['sum_sq'][1])
        print(f'Eta Quadrado Parcial para {coluna1} ~ {coluna2}: {eta_squared_partial:.4f}')

Eta Quadrado Parcial para NR_PERFIL ~ NR_VOTAVEL_1T: 0.0080
Eta Quadrado Parcial para NR_PERFIL ~ NR_VOTAVEL_2T: 0.0025
Eta Quadrado Parcial para NR_PERFIL ~ NR_BAIRRO: 0.0023
Eta Quadrado Parcial para NR_PERFIL ~ NR_GENERO: 0.9301
Eta Quadrado Parcial para NR_PERFIL ~ NR_GRAU_ESCOLARIDADE: 0.0135
Eta Quadrado Parcial para NR_PERFIL ~ NR_LOCAL_VOTACAO: 0.0571
Eta Quadrado Parcial para NR_PERFIL ~ NR_FAIXA_ETARIA: 0.0243
Eta Quadrado Parcial para NR_VOTAVEL_1T ~ NR_VOTAVEL_2T: 0.5829
Eta Quadrado Parcial para NR_VOTAVEL_1T ~ NR_BAIRRO: 0.1344
Eta Quadrado Parcial para NR_VOTAVEL_1T ~ NR_GENERO: 0.0013
Eta Quadrado Parcial para NR_VOTAVEL_1T ~ NR_GRAU_ESCOLARIDADE: 0.0239
Eta Quadrado Parcial para NR_VOTAVEL_1T ~ NR_LOCAL_VOTACAO: 0.0014
Eta Quadrado Parcial para NR_VOTAVEL_1T ~ NR_FAIXA_ETARIA: 0.0115
Eta Quadrado Parcial para NR_VOTAVEL_2T ~ NR_BAIRRO: 0.1105
Eta Quadrado Parcial para NR_VOTAVEL_2T ~ NR_GENERO: 0.0001
Eta Quadrado Parcial para NR_VOTAVEL_2T ~ NR_GRAU_ESCOLARIDADE: 0.01

Do relatório acima, podemos observar que o teste de associação feito com o Qui-Quadrado foi refletido na análise de explicabilidade, podendo inferir as mesmas questões sobre este.

# Conclusões

O intuíto da nossa análise, desde o início, consistia na resposta da seguinte pergunta: "Existe uma associação geográfica entre os socioindicadores de uma região da cidade de Campina Grande, os perfis de eleitores e os votos que dela emergem?".

Para embasar as conclusões, utilizamos uma pesquisa realizada pela Prefeitura Municipal de Campina Grande utilizando dados do Censo do IBGE de 2010, indicando quais regiões apresentavam os melhores e os piores socioindicadores. Para conversar com as conclusões dessa pesquisa, unimos uma análise dos votos emergidos para **presidente**, no primeiro e segundo turno das eleições de 2022, nas regiões da cidade e qual a distribuição dos perfis dos eleitores aptos a depositarem seus votos nas urnas correspondentes, com dados retirados dos Boletins de Urna disponibilizados pelo Tribunal Superior Eleitoral (TSE).

Da análise, podemos concluir que há certa associação entre os socioindicadores e os votos que emergem dos locais de votação, dado que o candidato **LULA** venceu em mais urnas localizadas em bairros mais afastados do centro da cidade e o candidato **JAIR BOLSONARO** ganhou em mais urnas nos bairros centralizados da cidade.  

Dos componentes dos perfis, a escolaridade foi a mais descritiva quando relacionada às zonas da cidade. Apesar da mais frequente, ENSINO MÉDIO COMPLETO, ser dispersa por toda a região da cidade, vemos uma concentração de eleitores com SUPERIOR COMPLETO por bairros com os maiores socioindicadores.

No último passo, as análises estatísticas com o teste do Qui-Quadrado e do Eta Quadrado Parcial, mostram certa relação entre variáveis interessantes, como o perfil e o bairro, votável e local de votação/bairro, apesar de não ter indicado alta explicabilidade entre o perfil do eleitor e o votável.

Por fim, podemos concluir da totalidade de nossa análise que há um comportamento relacionado entre socioindicadores e aspectos eleitorais dos bairros, que não emergem necessariamente de todos os possíveis indicadores, mas é perceptível em muitas características.
